http://finugorbib.com/bible/moksha/00_b_index_ru.html

http://finugorbib.com/bible/moksha/00_ot_index_na.html

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

## Parsing

In [2]:
base_url = 'http://finugorbib.com/bible/moksha/'

In [ ]:
all_pairs = []

for root in ['http://finugorbib.com/bible/moksha/00_b_index_ru.html', 'http://finugorbib.com/bible/moksha/00_ot_index_ru.html']:
    content_list = BeautifulSoup(requests.get(root).content.decode('utf-8'))
    for a in tqdm(content_list.findAll('a')):
        if 'href' not in str(a):
            continue
        href = a['href']
        if href.startswith('..'):
            continue
        if href.startswith('00'):
            continue
        href = href.replace('na.html', 'ru.html')
        
        print(href)
        url = base_url + href
        src = BeautifulSoup(requests.get(url).content.decode('utf-8'))
        table = src.find('table')
        if table is None:
            print('!!!no table!!!')
            continue
        for row in table.findAll('tr'):
            pair = []
            for td in row.findAll('td'):
                con = []
                for c in td.contents:
                    if isinstance(c, str):
                        con.append(c)
                    elif str(c) == '<br/>':
                        con.append('\n')
                    elif c.attrs.get('class') == ['verse']:
                        continue
                    else:
                        con.append(c.text)
                pair.append(''.join(con).strip())
            if len(pair) != 2:
                print(len(pair))
                continue
            all_pairs.append({
                'mdf': pair[0],
                'ru': pair[1],
                'source': url
            })

In [19]:
print(len(all_pairs))

12517


In [20]:
import random

In [21]:
random.choice(all_pairs)

{'mdf': 'Ливозьботмоса кармат сатнемонза кшицень,\n\xa0\xa0\xa0мзярс аф мрдат меки модати,\n\xa0\xa0\xa0конань эзда улеть сявф.\nПульста тон тифат\n\xa0\xa0\xa0и пульти меки мрдат».',
 'ru': 'в поте лица твоего будешь есть хлеб, доколе не возвратишься в землю, из которой ты взят, ибо прах ты и в прах возвратишься.',
 'source': 'http://finugorbib.com/bible/moksha/01_Gen03_ru.html'}

## Processing

1. В мокшанском тексте есть \n, а в русском этом одна и та же строка, поэтому заменяем '\n' на ' ' 
2. В мокшанском тексте бывают сноски формата '\*[а-я]'. Удалим их, при этом в некоторых случаях после сноски пропущен пробел, поэтому заменить эти сноски на ' ', чтобы не создать невалидное слово
3. в мокшанском тексте много кавычек, а это отдельный токен, удалим его, чтобы предложения были схожи с русскими

In [55]:
import re

for i, pair in enumerate(all_pairs):
    if re.search(r'[\xa0\n]+', pair['mdf']):
        # print(i, pair['mdf'])
        # print()
        pair['mdf'] = re.sub(r'[\n\xa0\s]+', ' ', pair['mdf']).strip()

    if re.search(r'\*[а-я]', pair['mdf']):
        # print(i, pair['mdf'])
        # print()
        pair['mdf'] = re.sub(r'\*[а-я]', ' ', pair['mdf']).strip()

    if re.search(r'["«»]', pair['mdf']):
        # print(i, pair['mdf'])
        # print()
        pair['mdf'] = re.sub(r'["«»]', ' ', pair['mdf'])

## Save

In [57]:
import pandas as pd

In [58]:
df = pd.DataFrame(all_pairs)

In [59]:
df.shape

(12517, 3)

In [61]:
df = df.drop_duplicates(subset=['ru', 'mdf'])

In [62]:
df.shape

(11650, 3)

In [63]:
df.to_csv('results/moksha_bible.tsv', sep='\t')